In [6]:
!pip install pandas splink nicknames


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd

import splink.comparison_library as cl
from splink import DuckDBAPI, Linker, SettingsCreator, block_on
from nicknames import NickNamer

nn = NickNamer()

data = [
    {"unique_id": 1, "first_name": "John", "last_name": "Smith"},
    {"unique_id": 2, "first_name": "John", "last_name": "Smiths"},
    {"unique_id": 3, "first_name": "Johnny", "last_name": "Smith"},
    {"unique_id": 4, "first_name": "Jonnie", "last_name": "Wilkins"},
    {"unique_id": 5, "first_name": "Peter", "last_name": "Smith"}
]

df = pd.DataFrame(data)


In [4]:
list_first_name_nicks=[]
list_last_name_nicks=[]

for index, row in df.iterrows():
    first_name = row['first_name']
    last_name = row['last_name']
    nicks = nn.nicknames_of(first_name)
    list_first_name_nicks.append(list(nicks))
    nicks = nn.nicknames_of(last_name)
    list_last_name_nicks.append(list(nicks))

df['nicks_first_name'] = list_first_name_nicks
df['nicks_last_name'] = list_last_name_nicks

In [5]:
print(df)

   unique_id first_name last_name  \
0          1       John     Smith   
1          2       John    Smiths   
2          3     Johnny     Smith   
3          4     Jonnie   Wilkins   
4          5      Peter     Smith   

                                nicks_first_name nicks_last_name  
0  [jonny, johnny, jock, jack, jon, jonnie, ian]        [smitty]  
1  [jonny, johnny, jock, jack, jon, jonnie, ian]              []  
2                                             []        [smitty]  
3                                             []              []  
4                                   [pete, pate]        [smitty]  


In [ ]:
settings = SettingsCreator(
    link_type="dedupe_only",
    blocking_rules_to_generate_predictions=[
        block_on("first_name"),
    ],
    comparisons=[
        cl.ExactMatch("first_name"),
        cl.CustomLevel("sql_condition": "list_contains(nicks_first_name, first_name_r) OR list_contains(nicknames_r, first_name_l)",
                      "label_for_charts": "Nickname match"),
    ]
)